In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("final_data_half.csv",low_memory=False)

In [10]:
from sklearn.model_selection import train_test_split

# Define the proportion of data for each set
test_size = 0.2  # 20% of the data for testing
val_size = 0.2   # 20% of the data for validation
train_size = 1 - (test_size + val_size)  # Remaining data for training

# Split the data into train-validation-test sets
train_val_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

# Calculate the adjusted validation size based on the remaining data after test split
adjusted_val_size = val_size / (train_size + val_size - test_size)

# Split the train-validation set into train and validation sets
train_df, val_df = train_test_split(train_val_df, test_size=adjusted_val_size, random_state=42)

In [12]:
train_df = pd.DataFrame(data=train_df)
val_df = pd.DataFrame(data=val_df)
test_df = pd.DataFrame(data=test_df)

train_df = train_df.dropna()
test_df = test_df.dropna()
val_df = val_df.dropna()

(107025, 11)

In [14]:
from sklearn.preprocessing import StandardScaler

feature_columns = ['station_id', 'year', 'month', 'day', 'hour',
       'occupation_1h_before', 'occupation_2h_before', 'occupation_3h_before',
       'occupation_4h_before']  
target_column = 'occupation'  

# Separate the features and the target
X_train = train_df[feature_columns]
y_train = train_df[target_column]

X_val = val_df[feature_columns]
y_val = val_df[target_column]

scaler = StandardScaler()

#Scale the train and validation
X_train = scaler.fit_transform(X_train)

X_val = scaler.transform(X_val)

In [17]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.metrics import mean_squared_error

# Create the neural network model
model = Sequential()
model.add(Dense(9,kernel_initializer='normal', 
                activation='relu', 
                input_dim=X_train.shape[1]))
model.add(Dense(1, kernel_initializer='normal'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mse'])

# Train the model
model.fit(X_train, y_train, 
          epochs=30, 
          batch_size=10, 
          verbose=1,
          validation_data=(X_val, y_val))

Epoch 1/30
21405/21405 [==============================] - 49s 2ms/step - loss: 0.0146 - mse: 0.0146 - val_loss: 0.0138 - val_mse: 0.0138
Epoch 2/30
21405/21405 [==============================] - 48s 2ms/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.0134 - val_mse: 0.0134
Epoch 3/30
21405/21405 [==============================] - 49s 2ms/step - loss: 0.0133 - mse: 0.0133 - val_loss: 0.0134 - val_mse: 0.0134
Epoch 4/30
21405/21405 [==============================] - 49s 2ms/step - loss: 0.0133 - mse: 0.0133 - val_loss: 0.0133 - val_mse: 0.0133
Epoch 5/30
21405/21405 [==============================] - 58s 3ms/step - loss: 0.0133 - mse: 0.0133 - val_loss: 0.0132 - val_mse: 0.0132
Epoch 6/30
21405/21405 [==============================] - 49s 2ms/step - loss: 0.0132 - mse: 0.0132 - val_loss: 0.0133 - val_mse: 0.0133
Epoch 7/30
21405/21405 [==============================] - 49s 2ms/step - loss: 0.0132 - mse: 0.0132 - val_loss: 0.0132 - val_mse: 0.0132
Epoch 8/30
21405/21405 [=================

In [18]:
# Make predictions on the test data

y_pred = model.predict(X_val)
# Evaluate the model's performance
mse = mean_squared_error(y_val, y_pred)

print("Mean Squared Error:", mse)


3345/3345 [==============================] - 5s 1ms/step
Mean Squared Error: 0.013517080549826184


In [19]:
# Validate the model with the test df
X_test = test_df[feature_columns]
y_test = test_df[target_column]

# Scale test data set
X_test = scaler.fit_transform(X_test)

y_test_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_test_pred)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)


2509/2509 [==============================] - 4s 2ms/step
Mean Squared Error (MSE): 0.0132684436566968
